In [4]:
import os
os.environ["OMP_NUM_THREADS"] = "1" 
os.environ["MKL_NUM_THREADS"] = "1" 

from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Laddar upp csv-fil 'wholesale_customerdata' 
customer_data = pd.read_csv('Wholesale customer sdata.csv')